In [3]:
# 저장된 데이터 파일 찾기 및 로드
import glob
import json
import pandas as pd
from datetime import datetime

# 가장 최신 JSON 파일 찾기
json_files = glob.glob('munpia_contest_ranking_*.json')
if json_files:
    latest_json = max(json_files, key=lambda x: x.split('_')[-1].replace('.json', ''))
    print(f"📁 최신 데이터 파일: {latest_json}")
    
    # JSON 데이터 로드
    with open(latest_json, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # DataFrame 생성
    df = pd.DataFrame(data['rankings'])
    
    print(f"✅ 데이터 로드 완료: {len(df)}개 작품")
    print(f"📋 컬럼: {list(df.columns)}")
    
    # 기본 정보 출력
    print(f"\n📊 기본 통계:")
    print(f"   제목이 있는 작품: {df['title'].notna().sum()}개")
    print(f"   조회수가 있는 작품: {df['view_count_number'].notna().sum()}개")
    print(f"   연독률이 있는 작품: {df['reading_rate_number'].notna().sum()}개")
    
else:
    print("❌ 저장된 데이터 파일을 찾을 수 없습니다.")
    print("   먼저 파서를 실행하여 데이터를 수집해주세요.")

📁 최신 데이터 파일: munpia_contest_ranking_20250605_225739.json
✅ 데이터 로드 완료: 200개 작품
📋 컬럼: ['crawl_rank', 'novel_url', 'novel_id', 'rank_number', 'author', 'title', 'genre', 'view_count', 'view_count_number', 'rank_change', 'rank_change_number', 'reading_rate', 'reading_rate_number']

📊 기본 통계:
   제목이 있는 작품: 200개
   조회수가 있는 작품: 200개
   연독률이 있는 작품: 152개


In [1]:
import json
import pandas as pd
from datetime import datetime
import re
from bs4 import BeautifulSoup
import os

class MunpiaImprovedParser:
    def __init__(self):
        self.url = "https://novel.munpia.com/page/hd.contest2025/group/contest/view/a/contest/list"
        self.html_file_path = r"E:\장난감 문피아\문피아0611.html"

    def parse_local_file(self):
        """로컬 HTML 파일 파싱"""
        try:
            if not os.path.exists(self.html_file_path):
                print(f"❌ 파일이 존재하지 않습니다: {self.html_file_path}")
                return None

            print(f"📁 로컬 파일 읽기: {self.html_file_path}")

            # 인코딩 처리
            html_content = None
            for encoding in ['utf-8', 'utf-8-sig', 'cp949', 'euc-kr']:
                try:
                    with open(self.html_file_path, 'r', encoding=encoding) as f:
                        html_content = f.read()
                    print(f"✅ 인코딩 성공: {encoding}")
                    break
                except:
                    continue

            if not html_content:
                print("❌ 파일 인코딩 실패")
                return None

            print(f"📄 HTML 크기: {len(html_content):,} 문자")

            return self.parse_html_content(html_content)

        except Exception as e:
            print(f"❌ 파일 처리 실패: {e}")
            return None

    def parse_html_content(self, html_content):
        """HTML 내용 파싱"""
        try:
            print("🔍 HTML 파싱 시작...")

            soup = BeautifulSoup(html_content, 'html.parser')

            # 페이지 제목 확인
            title = soup.find('title')
            if title:
                print(f"페이지 제목: {title.get_text()}")

            rankings = self.extract_rankings(soup)

            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            result = {
                'collection_time': current_time,
                'total_works': len(rankings),
                'rankings': rankings,
                'url': self.url,
                'method': 'local_file_parsing'
            }

            return result

        except Exception as e:
            print(f"❌ HTML 파싱 실패: {e}")
            import traceback
            traceback.print_exc()
            return None

    def extract_rankings(self, soup):
        """개선된 랭킹 데이터 추출"""
        rankings = []

        try:
            print("🔍 랭킹 데이터 검색 중...")

            # best-rank-list-display 컨테이너 찾기
            rank_container = soup.find('div', {'id': 'best-rank-list-display'})
            if not rank_container:
                print("❌ best-rank-list-display 컨테이너를 찾을 수 없습니다")
                return []

            print("✅ best-rank-list-display 컨테이너 발견")

            # 소설 링크들 찾기 (munpia.com/숫자 패턴)
            novel_links = []
            all_links = rank_container.find_all('a', href=True)

            for link in all_links:
                href = link.get('href', '')
                # novel.munpia.com/숫자 패턴 확인
                if re.search(r'novel\.munpia\.com/\d{6,}', href):
                    novel_links.append(link)

            print(f"📚 발견된 소설 링크: {len(novel_links)}개")

            if not novel_links:
                print("❌ 유효한 소설 링크를 찾을 수 없습니다")
                return []

            # 각 소설 링크에서 데이터 추출
            for idx, link in enumerate(novel_links, 1):
                try:
                    rank_data = self.extract_single_item(link, idx)
                    if rank_data:
                        rankings.append(rank_data)

                        # 처음 20개 미리보기
                        if idx <= 20:
                            rank_num = rank_data.get('rank_number', '?')
                            title = rank_data.get('title', 'N/A')[:30]
                            author = rank_data.get('author', 'N/A')[:15]
                            view_count = rank_data.get('view_count', 'N/A')
                            genre = rank_data.get('genre', 'N/A')[:20]
                            reading_rate = rank_data.get('reading_rate', 'N/A')

                            print(f"{rank_num:>3}. {title:<30} | {author:<15} | {view_count:>8} | {reading_rate:>7} | {genre}")

                except Exception as e:
                    print(f"항목 {idx} 처리 중 오류: {e}")
                    continue

            print(f"✅ 총 {len(rankings)}개 작품 데이터 추출 완료")
            return rankings

        except Exception as e:
            print(f"❌ 랭킹 추출 실패: {e}")
            import traceback
            traceback.print_exc()
            return []

    def extract_single_item(self, item, idx):
        """단일 항목 데이터 추출"""
        try:
            # URL 추출 및 정규화
            novel_url = item.get('href', '')
            if not novel_url.startswith('http'):
                novel_url = 'https://' + novel_url if novel_url.startswith('novel.munpia.com') else 'https://novel.munpia.com' + novel_url

            # 소설 ID 추출
            novel_id_match = re.search(r'novel\.munpia\.com/(\d+)', novel_url)
            novel_id = novel_id_match.group(1) if novel_id_match else None

            rank_data = {
                'crawl_rank': idx,
                'novel_url': novel_url,
                'novel_id': novel_id
            }

            # 직접 자식 div들에서 클래스별로 데이터 추출
            direct_divs = item.find_all('div', recursive=False)

            for div in direct_divs:
                div_classes = div.get('class', [])
                if not div_classes:
                    continue

                class_name = ' '.join(div_classes)

                # 순위 번호
                if 'num' in class_name:
                    rank_data['rank_number'] = self.clean_text(div.get_text())

                # 작가명
                elif 'author' in class_name:
                    rank_data['author'] = self.clean_text(div.get_text())

                # 제목
                elif 'title' in class_name:
                    title_span = div.find('span', class_='title-wrap')
                    if title_span:
                        rank_data['title'] = self.clean_text(title_span.get_text())
                    else:
                        # 다른 span이나 텍스트 확인
                        span = div.find('span')
                        if span:
                            rank_data['title'] = self.clean_text(span.get_text())
                        else:
                            rank_data['title'] = self.clean_text(div.get_text())

                # 장르
                elif 'genre' in class_name:
                    span = div.find('span')
                    if span:
                        rank_data['genre'] = self.clean_text(span.get_text())
                    else:
                        rank_data['genre'] = self.clean_text(div.get_text())

                # 조회수
                elif 'view-count' in class_name:
                    view_text = self.clean_text(div.get_text())
                    rank_data['view_count'] = view_text
                    rank_data['view_count_number'] = self.extract_number(view_text)

                # 순위 변동
                elif 'rank-changes' in class_name:
                    # SVG 태그 제거
                    for svg in div.find_all('svg'):
                        svg.decompose()

                    change_text = self.clean_text(div.get_text())
                    rank_data['rank_change'] = change_text

                    # 숫자 변동 추출
                    if change_text and re.match(r'^[+-]?\d+$', change_text.strip()):
                        try:
                            rank_data['rank_change_number'] = int(change_text.strip())
                        except:
                            pass

                # 연독률
                elif 'percent' in class_name:
                    span = div.find('span')
                    if span:
                        reading_rate = self.clean_text(span.get_text())
                    else:
                        reading_rate = self.clean_text(div.get_text())

                    rank_data['reading_rate'] = reading_rate

                    # 숫자로 변환
                    if reading_rate and '%' in reading_rate:
                        try:
                            rate_num = float(reading_rate.replace('%', '').strip())
                            if 0 <= rate_num <= 100:
                                rank_data['reading_rate_number'] = rate_num
                        except:
                            pass

            # 필수 필드 확인
            if rank_data.get('novel_id') and (rank_data.get('title') or rank_data.get('author')):
                return rank_data

            return None

        except Exception as e:
            print(f"항목 추출 중 오류: {e}")
            return None

    def clean_text(self, text):
        """텍스트 정리"""
        if not text:
            return ""

        text = re.sub(r'\s+', ' ', str(text))
        text = text.replace('\xa0', ' ')
        text = text.replace('\u200b', '')
        text = text.strip()

        return text

    def extract_number(self, text):
        """숫자 추출"""
        if not text:
            return 0

        # 콤마가 포함된 숫자 패턴
        numbers = re.findall(r'\d{1,3}(?:,\d{3})*', text)

        if numbers:
            try:
                # 가장 큰 숫자 반환
                max_num = 0
                for num_str in numbers:
                    try:
                        num = int(num_str.replace(',', ''))
                        if num > max_num:
                            max_num = num
                    except:
                        continue
                return max_num
            except:
                return 0

        return 0

    def save_data(self, data):
        """데이터 저장"""
        if not data or not data.get('rankings'):
            print("❌ 저장할 데이터가 없습니다.")
            return False

        try:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

            # JSON 저장
            json_filename = f"munpia_contest_ranking_{timestamp}.json"
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)

            # CSV 저장
            df = pd.DataFrame(data['rankings'])
            csv_filename = f"munpia_contest_ranking_{timestamp}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

            print(f"✅ 데이터 저장 완료:")
            print(f"   📄 JSON: {json_filename}")
            print(f"   📊 CSV: {csv_filename}")

            # 상세 통계
            rankings = data['rankings']
            print(f"\n📈 상세 통계:")
            print(f"   총 작품 수: {len(rankings)}")

            if rankings:
                # 필드별 추출 성공률
                fields_stats = {
                    'novel_id': len([r for r in rankings if r.get('novel_id')]),
                    'title': len([r for r in rankings if r.get('title')]),
                    'author': len([r for r in rankings if r.get('author')]),
                    'genre': len([r for r in rankings if r.get('genre')]),
                    'view_count': len([r for r in rankings if r.get('view_count')]),
                    'reading_rate': len([r for r in rankings if r.get('reading_rate')]),
                    'rank_change': len([r for r in rankings if r.get('rank_change')])
                }

                print("   필드별 추출 성공:")
                for field, count in fields_stats.items():
                    percentage = (count / len(rankings)) * 100
                    print(f"     {field}: {count}개 ({percentage:.1f}%)")

                # 조회수 통계
                view_counts = [r.get('view_count_number', 0) for r in rankings if r.get('view_count_number', 0) > 0]
                if view_counts:
                    print(f"\n   조회수 통계:")
                    print(f"     평균: {sum(view_counts)/len(view_counts):,.0f}")
                    print(f"     최고: {max(view_counts):,}")
                    print(f"     최저: {min(view_counts):,}")

                # 독서율 통계
                reading_rates = [r.get('reading_rate_number', 0) for r in rankings if r.get('reading_rate_number', 0) > 0]
                if reading_rates:
                    print(f"\n   독서율 통계:")
                    print(f"     평균: {sum(reading_rates)/len(reading_rates):.2f}%")
                    print(f"     최고: {max(reading_rates):.2f}%")
                    print(f"     최저: {min(reading_rates):.2f}%")

            print(f"\n💾 파일이 저장되었습니다:")
            print(f"   {os.path.abspath(json_filename)}")
            print(f"   {os.path.abspath(csv_filename)}")

            return True

        except Exception as e:
            print(f"❌ 저장 실패: {e}")
            import traceback
            traceback.print_exc()
            return False

In [2]:
# 개선된 파서 실행
print("🚀 문피아 개선된 로컬 파일 파서 시작")
print("="*60)
print()
print("📋 개선사항:")
print("   ✅ 로컬 HTML 파일 직접 읽기")
print("   ✅ 정확한 HTML 구조 분석")
print("   ✅ 필드별 정확한 추출")
print("   ✅ 조회수, 연독률 숫자 변환")
print("   ✅ 상세 통계 제공")
print()

parser = MunpiaImprovedParser()
print(f"📁 파일 경로: {parser.html_file_path}")
print()

result = parser.parse_local_file()

if result:
    success = parser.save_data(result)
    if success:
        print(f"\n🎉 파싱 완료! {result['total_works']}개 작품 데이터 추출 및 저장 성공")
    else:
        print(f"\n⚠️ 파싱은 성공했지만 저장 중 오류 발생")
else:
    print("\n❌ 파싱 실패")

🚀 문피아 개선된 로컬 파일 파서 시작

📋 개선사항:
   ✅ 로컬 HTML 파일 직접 읽기
   ✅ 정확한 HTML 구조 분석
   ✅ 필드별 정확한 추출
   ✅ 조회수, 연독률 숫자 변환
   ✅ 상세 통계 제공

📁 파일 경로: E:\장난감 문피아\문피아0611.html

📁 로컬 파일 읽기: E:\장난감 문피아\문피아0611.html
✅ 인코딩 성공: cp949
📄 HTML 크기: 286,889 문자
🔍 HTML 파싱 시작...
🔍 랭킹 데이터 검색 중...
✅ best-rank-list-display 컨테이너 발견
📚 발견된 소설 링크: 200개
  1. 방구석 경제학자                       | 페르세르크           |   10,022 |  63.18% | 현대판타지
  2. 뇌각성 후 인생 역전                    | 연함™             |    7,831 |  37.03% | 현대판타지, 판타지
  3. 자살 부대의 불사자는 착각당한다              | 풀드로우            |    7,437 |  63.21% | 퓨전, 현대판타지
  4. 백수가 너무 유능하다.                   | 시하              |    7,138 |  59.31% | 현대판타지, 드라마
  5. 하루에 2배씩 강해지는 헌터                | 캡슐호텔            |    6,427 |       - | 현대판타지, 퓨전
  6. 삼류무사에서 천억 투수까지                 | 우림™             |    5,519 |  42.22% | 스포츠, 현대판타지
  7. 초월급 투자자가 돈을 너무 잘 벎             | 늑타리요            |    5,449 |  61.73% | 현대판타지, 퓨전
  8. 알 카포네 검은머리 데릴사위                | Merkava         |    5,417 |  65.92% |